# Summary Of Science

In [26]:
#!pip install tensorflow
#!pip install transformers
#!pip install torch
#!pip install sentencepiece

In [1]:
import requests
import zipfile
import requests
import io
import json
import zipfile
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import seaborn as sns
import matplotlib.pyplot as plt
import html
from collections import defaultdict, Counter
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import ngrams
from rouge_score import rouge_scorer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import time
import torch
import random
import tensorflow
import sentencepiece

sns.set_style("dark")
plot_dims = (16, 16)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\derek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
DATASET_PATH = "./data/aan_data"

In [4]:
#load our 10k data into a dataframe
papers = []
filenames = [] #keep a reference for later transformations

abstracts, bodys, cits =[],[],[]

for root, dirs, files in os.walk(DATASET_PATH):
    for f in files:
        fn = root+"/"+f
        if "abstract" in fn:            
            in_file = open(fn, 'r')
            file = in_file.readlines()

            new_list = [''.join(file[i*4:(i+1)*4]) for i in range(int(len(file)/4))]
            list_no_n = [item.replace('\n','') for item in new_list]

            string = ''
            for item in list_no_n:
                string = string + item            
            abstracts.append(string)
            
        elif "body" in fn:
            in_file = open(fn, 'r')
            file = in_file.readlines()

            new_list = [''.join(file[i*4:(i+1)*4]) for i in range(int(len(file)/4))]
            list_no_n = [item.replace('\n','') for item in new_list]

            string = ''
            for item in list_no_n:
                string = string + item            
            bodys.append(string)
        
        elif "CS" in fn:
            in_file = open(fn, 'r')
            file = in_file.readlines()
            string = [''.join(file) for i in file]
            if len(string) > 0: 
                string = [''.join(file) for i in file][0].replace('\n','')
            cits.append(string)
            
        else:
            pass
        
        
        
        
    #    with open(fn) as jsonfile:
    #        d = json.load(jsonfile)
    #    papers.append(d)
    #    filenames.append(f)

df = pd.DataFrame({'Abstract':abstracts,'Body':bodys,'Citations':cits})
        

In [5]:
df

,Abstract,Body,Citations
0,A major obstacle to the construction ofa pro...,Parallel texts have been used in a number of...,A compilation of parallel texts offered in a s...
1,This paper proposes a way to improve the tran...,"Recently, various dialogue translation syste...",We plan to improve the accuracy obtained so fa...
2,This paper describes an application of APE (t...,The purpose of the Atlas project is to enlarg...,Computer dialogue is now used at production st...
3,In this paper we describe an implemented fram...,In this paper we present a linguistically mot...,"From this viewpoint, research on paraphrasing ..."
4,"This paper reports on a large-scale, end-to- ...",One major goal of information extraction (IE)...,"For example, Aone and Ramos-Santacruz (2000) p..."
...,...,...,...
4268,We present in this paper an unsupervised meth...,Development of electronic morphological resou...,Next along the spectrum of orthographic simila...
4269,This paper presents an unsupervised method fo...,Choosing the correct translation of a content...,"For comparable corpora, previous bilingual sen..."
4270,In this paper we report on an unsupervised a...,In this paper we discuss a potential solutio...,Watkinson and Manandhar (1999) present an unsu...
4271,This paper presents results from a study comp...,In evaluating the state of technology for ext...,"It may be noted that ""correctly"" is a problema..."


In [116]:
src_text = [
     """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
]

model_name = 'google/pegasus-xsum'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

In [118]:
tgt_text[0]

"California's largest electricity provider has turned off power to hundreds of thousands of customers."

## Baseline Pretrain

# AAN PEGASUS-XSUM

In [76]:

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL','rougeLsum'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')
for k in scores:
    score = scores[k]
    print("{} : {}".format(k,score[0]))

rouge1 : 0.75
rouge2 : 0.2857142857142857
rougeL : 0.625
rougeLsum : 0.625


In [27]:
df.Abstract[136]

" In this paper, we propose a method for ex- tracting key paragraphs in articles based on the degree of context dependency. Like Luhn's technique, our method assumes that the words related to theme in an arti- cle appear throughout paragraphs. Our ex- traction technique of keywords is based on the degree of context dependency that how strongly a word is related to a given con- text. The results of experiments demon- strate the applicability of our proposed "

In [125]:
summaries = []
k=0
for i in df.Body:
    k+=1
    batch = tokenizer(i, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    f = open('aan_data_xsum_summaries_r1.csv','a')
    f.write(tgt_text[0])
    f.write('\n') 
    f.close()
    
    summaries.append(tgt_text)
    if k % 10 == 1:
        print(k)

1
11
21
31
41
51
61
71
81
91
101
111
121
131
141
151
161
171
181
191
201
211
221
231
241
251
261
271
281
291
301
311
321
331
341
351
361
371
381
391
401
411
421
431
441
451
461
471
481
491
501
511
521
531
541
551
561
571
581
591
601
611
621
631
641
651
661
671
681
691
701
711
721
731
741
751
761
771
781
791
801
811
821
831
841
851
861
871
881
891
901
911
921
931
941
951
961
971
981
991
1001
1011
1021
1031
1041
1051
1061
1071
1081
1091
1101
1111
1121
1131
1141
1151
1161
1171
1181
1191
1201
1211
1221
1231
1241
1251
1261
1271
1281
1291
1301
1311
1321
1331
1341
1351
1361
1371
1381
1391
1401
1411
1421
1431
1441
1451
1461
1471
1481
1491
1501
1511
1521
1531
1541
1551
1561
1571
1581
1591
1601
1611
1621
1631
1641
1651
1661
1671
1681
1691
1701
1711
1721
1731
1741
1751
1761
1771
1781


KeyboardInterrupt: 

In [126]:
len(summaries)

1781

In [127]:
summaries[29:39]

[['Text-to-speech (MT) systems have been around since the 1970s, but their performance has been poor.'],
 ['The aim of this paper is to identify the central factors that in- fluence pronominalization across genres.'],
 ['The aim of this project is to develop methods for segmenting written text in a collection to improve information retrieval.'],
 ['We present a word-for-word glossing algorithm that requires only a source language corpus.'],
 ['We have developed a language model that combines syntactic, semantic, and domain knowledge.'],
 ['dependency analysis is one of the most popular approaches to syntactic analysis in many languages.'],
 ['parsing and machine translation of low density languages are often in short supply.'],
 ['In this paper, we study a learning approach that is specifically tailored for problems in which the potential number of features is very large but only a fairly small number of them actually participates in the decision.'],
 ['We have developed a statistical 

In [128]:
df.Citations[39]

'This method was shown to be a generalization of the reference distribution method (Johnson and Riezler, 2000). Johnson and Riezler (2000) suggested the possibility of the method for adapting a stochastic unification-based grammar including HPSG to another domain. This model is introduced as a reference distribution (Jelinek, 1998; Johnson and Riezler, 2000) of the probabilistic HPSG model; i.e., the computation of parse trees given low probabilities by the model is omitted in the estimation stage (Miyao and Tsujii, 2005), or a probabilistic model can be augmented by several distributions estimated from the larger and simpler corpus (Johnson and Riezler, 2000). Johnson and Riezler (2000) achieved a gain of 1% over this result by including a classbased lexicalization. Johnson and Riezler (2000) looked at adding features to a maximum entropy model for stochastic unification-based grammars (SUBG), from corpora that are not annotated with the SUBG, but rather with simpler treebank annotati

In [129]:
summaries_downstream = []
k=0
for i in range(len(summaries)):
    string = summaries[i][0] + " " + df.Citations[i]
    k+=1
    
    batch = tokenizer(string, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

    f = open('aan_data_xsum_summaries_downstream_r1.csv','a')
    f.write(tgt_text[0])
    f.write('\n') 
    f.close()
    
    summaries_downstream.append(tgt_text)
    if k % 10 == 1:
        print(k)
summaries_downstream 

1
11
21
31
41
51
61
71
81
91
101
111
121
131
141
151
161
171
181
191
201
211
221
231
241
251
261
271
281
291
301
311
321
331
341
351
361
371
381
391
401
411
421
431
441
451
461
471
481
491
501
511
521
531
541
551
561
571
581
591
601
611
621
631
641
651
661
671
681
691
701
711
721
731
741
751
761
771
781
791
801
811
821
831
841
851
861
871
881
891
901
911
921
931
941
951
961
971
981
991
1001
1011
1021
1031
1041
1051
1061
1071
1081
1091
1101
1111
1121
1131
1141
1151
1161
1171
1181
1191
1201
1211
1221
1231
1241
1251
1261
1271
1281
1291
1301
1311
1321
1331
1341
1351
1361
1371
1381
1391
1401
1411
1421
1431
1441
1451
1461
1471
1481
1491
1501
1511
1521
1531
1541
1551
1561
1571
1581
1591
1601
1611
1621
1631
1641
1651
1661
1671
1681
1691
1701
1711
1721
1731
1741
1751
1761
1771
1781


[['A parallel corpus is a collection of parallel texts offered in a serviceable form.'],
 ['The use of "polite" people for dialogue translation has been proposed as a way to improve the accuracy of translation systems.'],
 ['We have developed a dialogue management system for intelligent tutoring systems called Atlas.'],
 ['This paper presents a corpus-based technique for paraphrasing Korean phrases and sentences into English structures.'],
 ['In this paper, we introduce a new type of information extraction system for extracting relations and events.'],
 ['Mixed-initiative dialogue systems (MIMIC) have been proposed as an alternative to directed dialog systems.'],
 ['The goal of this paper is to improve the performance of spoken dialogue systems.'],
 ['A formal representation language for natural language generation systems has been proposed in this paper.'],
 ['A prototype of a computer aided translation tool has been developed in our laboratory.'],
 ['The goal of this paper is to deve

In [134]:
#compare summaries
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL','rougeLsum'], use_stemmer=True)
scores = []
for i in range(len(summaries)):

    s = scorer.score(df.Abstract[i], summaries[i][0])
    
    #simpler flat structure
    x = {
        "rouge1:precision":s["rouge1"][0],
        "rouge1:recall":s["rouge1"][1],
        "rouge1:fmeasure":s["rouge1"][2],
        "rouge2:precision":s["rouge2"][0],
        "rouge2:recall":s["rouge2"][1],
        "rouge2:fmeasure":s["rouge2"][2],
        "rougeL:precision":s["rougeL"][0],
        "rougeL:recall":s["rougeL"][1],
        "rougeL:fmeasure":s["rougeL"][2],
        "rougeLsum:precision":s["rougeLsum"][0],
        "rougeLsum:recall":s["rougeLsum"][1],
        "rougeLsum:fmeasure":s["rougeLsum"][2],
        
    }
    scores.append(x)
    

In [135]:
scores_df = pd.DataFrame(scores)
scores_df.describe()

,rouge1:precision,rouge1:recall,rouge1:fmeasure,rouge2:precision,rouge2:recall,rouge2:fmeasure,rougeL:precision,rougeL:recall,rougeL:fmeasure,rougeLsum:precision,rougeLsum:recall,rougeLsum:fmeasure
count,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000
mean,0.552497,0.112506,0.178467,0.164739,0.031822,0.050710,0.398247,0.080006,0.127019,0.398247,0.080006,0.127019
std,0.184309,0.064809,0.082544,0.161686,0.040516,0.056609,0.156621,0.049813,0.063122,0.156621,0.049813,0.063122
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.428571,0.070707,0.123077,0.050000,0.008403,0.014706,0.285714,0.049689,0.086957,0.285714,0.049689,0.086957
50%,0.555556,0.098361,0.166667,0.125000,0.021277,0.035294,0.384615,0.068966,0.115942,0.384615,0.068966,0.115942
75%,0.681818,0.139535,0.222222,0.250000,0.043478,0.071429,0.500000,0.097087,0.152174,0.500000,0.097087,0.152174
max,1.000000,0.641026,0.746269,1.000000,0.552632,0.646154,1.000000,0.615385,0.716418,1.000000,0.615385,0.716418


In [136]:
#compare summaries
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL','rougeLsum'], use_stemmer=True)
scores = []
for i in range(len(summaries_downstream)):

    s = scorer.score(df.Abstract[i], summaries_downstream[i][0])
    
    #simpler flat structure
    x = {
        "rouge1:precision":s["rouge1"][0],
        "rouge1:recall":s["rouge1"][1],
        "rouge1:fmeasure":s["rouge1"][2],
        "rouge2:precision":s["rouge2"][0],
        "rouge2:recall":s["rouge2"][1],
        "rouge2:fmeasure":s["rouge2"][2],
        "rougeL:precision":s["rougeL"][0],
        "rougeL:recall":s["rougeL"][1],
        "rougeL:fmeasure":s["rougeL"][2],
        "rougeLsum:precision":s["rougeLsum"][0],
        "rougeLsum:recall":s["rougeLsum"][1],
        "rougeLsum:fmeasure":s["rougeLsum"][2],
        
    }
    scores.append(x)
    

In [137]:
scores_df = pd.DataFrame(scores)
scores_df.describe()

,rouge1:precision,rouge1:recall,rouge1:fmeasure,rouge2:precision,rouge2:recall,rouge2:fmeasure,rougeL:precision,rougeL:recall,rougeL:fmeasure,rougeLsum:precision,rougeLsum:recall,rougeLsum:fmeasure
count,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000
mean,0.492745,0.094913,0.152176,0.106582,0.018753,0.030466,0.355801,0.067263,0.108079,0.355801,0.067263,0.108079
std,0.170404,0.050522,0.066883,0.119133,0.023970,0.036138,0.138073,0.036348,0.048105,0.138073,0.036348,0.048105
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.380952,0.060345,0.105263,0.000000,0.000000,0.000000,0.260870,0.043103,0.074766,0.260870,0.043103,0.074766
50%,0.500000,0.084507,0.142857,0.076923,0.012821,0.021858,0.350000,0.060000,0.100719,0.350000,0.060000,0.100719
75%,0.600000,0.118644,0.188679,0.153846,0.025641,0.043478,0.437500,0.082645,0.133333,0.437500,0.082645,0.133333
max,1.000000,0.434783,0.433962,1.000000,0.272727,0.357143,1.000000,0.434783,0.400000,1.000000,0.434783,0.400000


# AAN PEGASUS-ARXIV

In [138]:
df

,Abstract,Body,Citations
0,A major obstacle to the construction ofa pro...,Parallel texts have been used in a number of...,A compilation of parallel texts offered in a s...
1,This paper proposes a way to improve the tran...,"Recently, various dialogue translation syste...",We plan to improve the accuracy obtained so fa...
2,This paper describes an application of APE (t...,The purpose of the Atlas project is to enlarg...,Computer dialogue is now used at production st...
3,In this paper we describe an implemented fram...,In this paper we present a linguistically mot...,"From this viewpoint, research on paraphrasing ..."
4,"This paper reports on a large-scale, end-to- ...",One major goal of information extraction (IE)...,"For example, Aone and Ramos-Santacruz (2000) p..."
...,...,...,...
4268,We present in this paper an unsupervised meth...,Development of electronic morphological resou...,Next along the spectrum of orthographic simila...
4269,This paper presents an unsupervised method fo...,Choosing the correct translation of a content...,"For comparable corpora, previous bilingual sen..."
4270,In this paper we report on an unsupervised a...,In this paper we discuss a potential solutio...,Watkinson and Manandhar (1999) present an unsu...
4271,This paper presents results from a study comp...,In evaluating the state of technology for ext...,"It may be noted that ""correctly"" is a problema..."


In [181]:
src_text = [
     """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
]

model_name = 'google/pegasus-cnn_dailymail'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
amodel = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = amodel.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
tgt_text

['PG&E scheduled the blackouts in response to forecasts for high winds amid dry conditions.<n>The aim is to reduce the risk of wildfires.']

In [ ]:
summaries_arxvi = []
k=0
for i in df.Body[0:len(summaries)]:
    k+=1
    batch = tokenizer(i, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = amodel.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    f = open('aan_data_cnn_dailymail_summaries_r1.csv','a')
    f.write(tgt_text[0])
    f.write('\n') 
    f.close()
    
    summaries_arxvi.append(tgt_text)
    if k % 10 == 1:
        print(k)

1
11
21
31
41
51
61
71
81
91
101
111
121
131
141
151
161
171
181
191
201
211
221
231
241
251
261
271
281
291
301
311
321
331
341
351
361
371
381
391
401
411
421
431
441
451
461
471
481
491
501
511
521
531
541
551
561
571
581
591
601
611
621
631
641
651
661
671
681
691
701
711
721
731
741
751
761
771
781
791


In [7]:
cnn_dm = pd.read_csv('aan_data_cnn_dailymail_summaries_r1.csv')
sum_cnndm = []
for i in cnn_dm['Summary']:
    sum_cnndm.append(i.replace('א',','))
sum_cnndm

['We will describe a method which automatically searches for parallel texts on the Web.<n>We will discuss the text mining algorithm we adopted, some issues in trans- lation model training using the generated parallel corpus.',
 'This paper describes a method of "polite- ness" selection according to a participant\'s so- cial role.<n>It is possible to use a "participant\'s so- cial role" to appropriately make the translation results "polite"',
 'The Atlas project is to enlarge the scope of student interaction in an intelligent tutoring system.<n>A key component of Atlas is APE, the Atlas Planning Engine.<n>APE could also be used to manage other types of human-computer conversation.',
 'We present a linguistically motivated framework for uniform lexico- structural processing.<n>It has been used for transformations of conceptual and syntactic structures during generation i monolingual nd multilingual natural language generation (NLG) and for transfer in machine translation (MT)<n>Our work 

In [9]:
#compare summaries
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL','rougeLsum'], use_stemmer=True)
scores = []
for i in range(len(sum_cnndm)):

    s = scorer.score(df.Abstract[i], sum_cnndm[i][0])
    
    #simpler flat structure
    x = {
        "rouge1:precision":s["rouge1"][0],
        "rouge1:recall":s["rouge1"][1],
        "rouge1:fmeasure":s["rouge1"][2],
        "rouge2:precision":s["rouge2"][0],
        "rouge2:recall":s["rouge2"][1],
        "rouge2:fmeasure":s["rouge2"][2],
        "rougeL:precision":s["rougeL"][0],
        "rougeL:recall":s["rougeL"][1],
        "rougeL:fmeasure":s["rougeL"][2],
        "rougeLsum:precision":s["rougeLsum"][0],
        "rougeLsum:recall":s["rougeLsum"][1],
        "rougeLsum:fmeasure":s["rougeLsum"][2],
        
    }
    scores.append(x)

In [10]:
scores_df = pd.DataFrame(scores)
scores_df.describe()

,rouge1:precision,rouge1:recall,rouge1:fmeasure,rouge2:precision,rouge2:recall,rouge2:fmeasure,rougeL:precision,rougeL:recall,rougeL:fmeasure,rougeLsum:precision,rougeLsum:recall,rougeLsum:fmeasure
count,817.000000,817.000000,817.000000,817.0,817.0,817.0,817.000000,817.000000,817.000000,817.000000,817.000000,817.000000
mean,0.149327,0.001536,0.003037,0.0,0.0,0.0,0.149327,0.001536,0.003037,0.149327,0.001536,0.003037
std,0.356629,0.004016,0.007921,0.0,0.0,0.0,0.356629,0.004016,0.007921,0.356629,0.004016,0.007921
min,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,0.024390,0.047619,0.0,0.0,0.0,1.000000,0.024390,0.047619,1.000000,0.024390,0.047619


In [12]:
src_text = [
     """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
]

model_name = 'google/pegasus-xsum'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

In [13]:
summar = pd.read_csv('aan_data_xsum_summaries_r1.csv')
summar2 = pd.read_csv('aan_data_xsum_summaries_downstream_r1.csv')
cnn_dm = pd.read_csv('aan_data_cnn_dailymail_summaries_r1.csv')
s1,s2,sum_cnndm = [],[],[]
for i in cnn_dm['Summary']:
    sum_cnndm.append(i.replace('א',','))
for i in summar['Summary']:
    s1.append(i.replace('א',','))
for i in summar2['Summary']:
    s2.append(i.replace('א',','))

In [14]:
summaries_downstream_combined2 = []
k=0
for i in range(len(sum_cnndm)):
    string = s1[i] + " " + s2[i] + " " + sum_cnndm[i] + " " + df.Citations[i]
    k+=1
    
    batch = tokenizer(string, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

    f = open('aan_data_xsum_and_cnndailymail_summaries_downstream_r2.csv','a')
    f.write(tgt_text[0])
    f.write('\n') 
    f.close()
    
    summaries_downstream_combined2.append(tgt_text)
    if k % 10 == 1:
        print(k)
summaries_downstream_combined2

1
11
21
31
41
51
61
71
81
91
101
111
121
131
141
151
161
171
181
191
201
211
221
231
241
251
261
271
281
291
301
311
321
331
341
351
361
371
381
391
401
411
421
431
441
451
461
471
481
491
501
511
521
531
541
551
561
571
581
591
601
611
621
631
641
651
661
671
681
691
701
711
721
731
741
751
761
771
781
791
801
811


[['A parallel corpus is a collection of parallel texts offered in a serviceable form.'],
 ['The use of "polite" people for dialogue translation has been proposed as a way to improve the accuracy of translation systems.'],
 ['We have developed a dialogue management system for intelligent tutoring systems called Atlas.'],
 ['corpus-based techniques for paraphrasing Korean phrases and sentences into English structures.'],
 ['In this paper, we introduce a new type of information extraction system for extracting relations and events.'],
 ['Dialogue systems have been proposed as an alternative to directed dialog systems.'],
 ['The aim of this paper is to improve the performance of spoken dialogue systems.'],
 ['A formal representation language for natural language generation systems has been proposed in this paper.'],
 ['A prototype of a computer aided translation tool has been developed in our laboratory.'],
 ['The goal of this paper is to develop a machine translation system that predicts 

In [24]:
#compare summaries
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL','rougeLsum'], use_stemmer=True)
scores = []
for i in range(len(summaries_downstream_combined2)):

    s = scorer.score(df.Abstract[i], summaries_downstream_combined2[i][0])
    
    #simpler flat structure
    x = {
        "rouge1:precision":s["rouge1"][0],
        "rouge1:recall":s["rouge1"][1],
        "rouge1:fmeasure":s["rouge1"][2],
        "rouge2:precision":s["rouge2"][0],
        "rouge2:recall":s["rouge2"][1],
        "rouge2:fmeasure":s["rouge2"][2],
        "rougeL:precision":s["rougeL"][0],
        "rougeL:recall":s["rougeL"][1],
        "rougeL:fmeasure":s["rougeL"][2],
        "rougeLsum:precision":s["rougeLsum"][0],
        "rougeLsum:recall":s["rougeLsum"][1],
        "rougeLsum:fmeasure":s["rougeLsum"][2],
        
    }
    scores.append(x)

In [25]:
scores_df = pd.DataFrame(scores)
scores_df.describe()

,rouge1:precision,rouge1:recall,rouge1:fmeasure,rouge2:precision,rouge2:recall,rouge2:fmeasure,rougeL:precision,rougeL:recall,rougeL:fmeasure,rougeLsum:precision,rougeLsum:recall,rougeLsum:fmeasure
count,817.000000,817.000000,817.000000,817.000000,817.000000,817.000000,817.000000,817.000000,817.000000,817.000000,817.000000,817.000000
mean,0.509185,0.096425,0.154848,0.122177,0.022147,0.035571,0.370250,0.068906,0.110859,0.370250,0.068906,0.110859
std,0.164687,0.055432,0.071397,0.139990,0.030686,0.044920,0.139383,0.041426,0.053538,0.139383,0.041426,0.053538
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.394737,0.059880,0.105263,0.000000,0.000000,0.000000,0.272727,0.043011,0.075472,0.272727,0.043011,0.075472
50%,0.500000,0.085106,0.144330,0.086957,0.013889,0.022989,0.357143,0.059701,0.100840,0.357143,0.059701,0.100840
75%,0.611111,0.120690,0.195652,0.166667,0.028846,0.048000,0.444444,0.081395,0.132701,0.444444,0.081395,0.132701
max,1.000000,0.434783,0.448980,1.000000,0.272727,0.425532,1.000000,0.434783,0.448980,1.000000,0.434783,0.448980


In [22]:
df.Citations[136]

'2 Domain Dependency of Words The domain dependency of words that how strongly a word features a given set of data (documents) contributes to event extraction, as we previously reported (Fukumoto et al.: 1997).'

In [11]:
sum_cnndm[0:6]

['We will describe a method which automatically searches for parallel texts on the Web.<n>We will discuss the text mining algorithm we adopted, some issues in trans- lation model training using the generated parallel corpus.',
 'This paper describes a method of "polite- ness" selection according to a participant\'s so- cial role.<n>It is possible to use a "participant\'s so- cial role" to appropriately make the translation results "polite"',
 'The Atlas project is to enlarge the scope of student interaction in an intelligent tutoring system.<n>A key component of Atlas is APE, the Atlas Planning Engine.<n>APE could also be used to manage other types of human-computer conversation.',
 'We present a linguistically motivated framework for uniform lexico- structural processing.<n>It has been used for transformations of conceptual and syntactic structures during generation i monolingual nd multilingual natural language generation (NLG) and for transfer in machine translation (MT)<n>Our work 

In [171]:
summaries[0:6]

[['In this paper, we will describe a method which automatically searches for parallel texts on the Web.'],
 ['The aim of this paper is to develop a method for selecting "polite" participants for dialogue translation systems.'],
 ['We have developed a dialogue management system for intelligent tutoring systems called Atlas.'],
 ['Natural language processing (NLP) and machine translation (MT) are two of the fastest growing areas of research in computer science.'],
 ['The Relation and Event Extraction System (REES) is a state-of-the-art information extraction system for extracting relations and events.'],
 ['In this paper, we describe a spoken dialogue system that automatically adapts dialogue strategies based on actual dialogue interactions.']]

In [170]:
summaries_arxvi[0:6]

[['We will describe a method which automatically searches for parallel texts on the Web.<n>We will discuss the text mining algorithm we adopted, some issues in trans- lation model training using the generated parallel corpus.'],
 ['This paper describes a method of "polite- ness" selection according to a participant\'s so- cial role.<n>It is possible to use a "participant\'s so- cial role" to appropriately make the translation results "polite"'],
 ['The Atlas project is to enlarge the scope of student interaction in an intelligent tutoring system.<n>A key component of Atlas is APE, the Atlas Planning Engine.<n>APE could also be used to manage other types of human-computer conversation.'],
 ['We present a linguistically motivated framework for uniform lexico- structural processing.<n>It has been used for transformations of conceptual and syntactic structures during generation i monolingual nd multilingual natural language generation (NLG) and for transfer in machine translation (MT)<n>Ou

In [ ]:
summaries_arxvi_downstream = []
k=0
for i in range(len(summaries_arxvi)):
    string = summaries_arxvi[i][0] + " " + df.Citations[i]
    k+=1
    
    batch = tokenizer(string, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = amodel.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

    f = open('aan_data_arxiv_summaries_downstream_r1.csv','a')
    f.write(tgt_text[0])
    f.write('\n') 
    f.close()
    
    summaries_arxvi_downstream.append(tgt_text)
    if k % 10 == 1:
        print(k)
summaries_arxvi_downstream 

In [178]:
src_text = [
     """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
]

model_name = 'google/pegasus-reddit_tifu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
bmodel = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = bmodel.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
tgt_text

['high temperatures and low relative humidity in the western united states over the last few days have resulted in widespread wildfire risk. <n> utility companies in several states have been activated to respond to power shutoffs caused by drought. in illinois, <n> the state public utility commission has declared a state of emergency for all new power shutoffs caused by drought. in the united states, <n> the national institute of environmental health sciences ( niehs ) has declared a state of emergency for all new power shutoffs caused by drought. in the united states <n>, the utility commission has also declared a state of emergency for all new power shutoffs caused by drought. <n> the niehs has declared a state of emergency for all new power shutoffs caused by drought in illinois, missouri, kentucky, pennsylvania, and iowa. <n> the niehs has also declared a state of emergency for all new power shutoffs caused by drought in minnesota. in the united states, <n> the national institute o

In [179]:
summaries_pub = []
k=0
for i in df.Body[0:6]:
    k+=1
    batch = tokenizer(i, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = bmodel.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    #f = open('aan_data_cnn_dailymail_summaries_r1.csv','a')
    #f.write(tgt_text[0])
    #f.write('\n') 
    #f.close()
    
    summaries_pub.append(tgt_text)
    if k % 10 == 1:
        print(k)

1


In [180]:
summaries_pub

[['the aim of this paper is to describe a new method of text mining, which is based on the inked model. <n> inked model is a probabilistic model, which is used to estimate the odds of a text to be found in another language. <n> the aim of this paper is to describe a new method of text mining, which is based on the inked model. <n> it is a probabilistic model, which is used to estimate the odds of a text to be found in another language. <n> the aim of this paper is to describe a new method of text mining, which is based on the inked model. <n> it is a probabilistic model, which is used to estimate the odds of a text to be found in another language. <n> the aim of this paper is to describe a new method of text mining, which is based on the inked model. <n> it is a probabilistic model, which is used to estimate the odds of a text to be found in another language.imagesfigure 1.figure 2.figure 3.figure 4.figure 5.figure 6.figure 7.figure 8.figure 9.figure 10.figure 11.figure 12.figure 13.fi

## Baseline pegasus-xsum results


In [9]:
src_text = [
     """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
]

model_name = 'google/pegasus-xsum'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

In [11]:
tgt_text

['The superfield formalism of the Becchi-Rouet-StoraTyutin (BRST) and anti-BRST symmetry in the Lagrangian densities of the four (3 + 1)-dimensional (4D) (non-)Abelian 1-form gauge theories within the framework of the superfield']

In [10]:
#generate xsum summaries
ts = time.time()
summaries = []
limit = 10000
for i,p in enumerate(papers):
    
    #skip inference on already summarized papers
    f = filenames[i]
    if os.path.exists("data/xsum/"+f):
        #load summary from disk
        with open("data/xsum/"+f) as inf:
            tgt = json.load(inf)
            summaries.append(tgt)
        continue
    
    #body_sents = p["fulltext"].split("\n\n")
    bts = time.time()
    #start with a batch size of 1
    batch = tokenizer(p["fulltext"], truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    #save the summary to disk
    with open("data/xsum/"+f, "w") as out:
        json.dump(tgt_text, out)
    
    summaries.append(tgt_text)
    bte = time.time()
    
    
te = time.time()
    

FileNotFoundError: [Errno 2] No such file or directory: 'data/xsum/arxiv.0704.0064.json'

In [18]:
print(len(summaries))

9998


In [19]:
print("Average of {} seconds per summary".format(int(te-ts)/len(summaries)))

Average of 1.7711542308461692 seconds per summary


In [21]:
idx = 1
print(summaries[idx])
print("***")
print(papers[idx]["summary"])

['Skewed parton distributions for simple, model wave-functions in a truncated two-body Fock space.']
***
The basic mechanism responsible for the widespread condensation of MgS in the
outflows from carbon rich stars on the tip of the AGB is discussed with the aim
of developing a condensation model that can be applied in model calculations of
dust formation in stellar winds.
  The different possibilities how MgS may be formed in the chemical environment
of outflows from carbon stars are explored by some thermochemical calculations
and by a detailed analysis of the growth kinetics of grains in stellar winds.
The optical properties of core-mantle grains with a MgS mantle are calculated
to demonstrate that such grains reproduce the structure of the observed 30
$\mu$m feature. These considerations are complemented by model calculations of
circumstellar dust shells around carbon stars.
  It is argued that MgS is formed via precipitation on silicon carbide grains.
This formation mechanism expl

In [77]:
#compare summaries
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL','rougeLsum'], use_stemmer=True)
scores = []
for i,p in enumerate(papers):
    if i >= len(summaries):
        break
    s = scorer.score(p["summary"], summaries[i][0])
    
    #simpler flat structure
    x = {
        "rouge1:precision":s["rouge1"][0],
        "rouge1:recall":s["rouge1"][1],
        "rouge1:fmeasure":s["rouge1"][2],
        "rouge2:precision":s["rouge2"][0],
        "rouge2:recall":s["rouge2"][1],
        "rouge2:fmeasure":s["rouge2"][2],
        "rougeL:precision":s["rougeL"][0],
        "rougeL:recall":s["rougeL"][1],
        "rougeL:fmeasure":s["rougeL"][2],
        "rougeLsum:precision":s["rougeLsum"][0],
        "rougeLsum:recall":s["rougeLsum"][1],
        "rougeLsum:fmeasure":s["rougeLsum"][2],
        
    }
    scores.append(x)
    

In [78]:
scores[0]

{'rouge1:precision': 0.21739130434782608,
 'rouge1:recall': 0.024875621890547265,
 'rouge1:fmeasure': 0.044642857142857144,
 'rouge2:precision': 0.0,
 'rouge2:recall': 0.0,
 'rouge2:fmeasure': 0.0,
 'rougeL:precision': 0.17391304347826086,
 'rougeL:recall': 0.01990049751243781,
 'rougeL:fmeasure': 0.03571428571428571,
 'rougeLsum:precision': 0.21739130434782608,
 'rougeLsum:recall': 0.024875621890547265,
 'rougeLsum:fmeasure': 0.044642857142857144}

In [79]:
scores_df = pd.DataFrame(scores)

In [80]:
scores_df.describe()

,rouge1:precision,rouge1:recall,rouge1:fmeasure,rouge2:precision,rouge2:recall,rouge2:fmeasure,rougeL:precision,rougeL:recall,rougeL:fmeasure,rougeLsum:precision,rougeLsum:recall,rougeLsum:fmeasure
count,2002.000000,2002.000000,2002.000000,2002.000000,2002.000000,2002.000000,2002.000000,2002.000000,2002.000000,2002.000000,2002.000000,2002.000000
mean,0.690436,0.144746,0.222224,0.455679,0.097808,0.148836,0.594981,0.126543,0.193401,0.659299,0.138010,0.211909
std,0.300222,0.133742,0.161530,0.342651,0.128792,0.162442,0.302202,0.131228,0.160169,0.299202,0.130872,0.158718
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.500000,0.067335,0.118829,0.136364,0.020051,0.034859,0.375000,0.053691,0.094250,0.461538,0.063202,0.112676
50%,0.777778,0.113980,0.191781,0.411765,0.056864,0.098200,0.607143,0.089286,0.150741,0.727273,0.107007,0.181534
75%,0.944444,0.177597,0.285714,0.777778,0.123263,0.203580,0.875000,0.151465,0.244847,0.915761,0.168675,0.269741
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [81]:
idx = 5
print(summaries[idx])
print("*"*80)
print(papers[idx]["summary"])

['We prove that weighted majority functions of n independent unbiased 1-valued variables are uniformly stable under noise.']
********************************************************************************
Benjamini, Kalai and Schramm (2001) showed that weighted majority functions
of $n$ independent unbiased bits are uniformly stable under noise: when each
bit is flipped with probability $\epsilon$, the probability $p_\epsilon$ that
the weighted majority changes is at most $C\epsilon^{1/4}$. They asked what is
the best possible exponent that could replace 1/4. We prove that the answer is
1/2. The upper bound obtained for $p_\epsilon$ is within a factor of
$\sqrt{\pi/2}+o(1)$ from the known lower bound when $\epsilon \to 0$ and
$n\epsilon\to \infty$.


## Baseline pegasus-arxiv results



In [22]:
src_text = [
     """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
]

model_name = 'google/pegasus-arxiv'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
amodel = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = amodel.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

In [ ]:
#generate summaries
ts = time.time()
asummaries = []
limit = 1000
for i,p in enumerate(papers):
    #body_sents = p["fulltext"].split("\n\n")
    
    #skip inference on already summarized papers
    f = filenames[i]
    if os.path.exists("data/pegasus_arxiv/"+f):
        #load summary from disk
        with open("data/pegasus_arxiv/"+f) as inf:
            tgt = json.load(inf)
            asummaries.append(tgt)
        continue
    
    bts = time.time()
    #start with a batch size of 1
    batch = tokenizer(p["fulltext"], truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = amodel.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    asummaries.append(tgt_text)
    bte = time.time()
    
    #save the summary to disk
    with open("data/pegasus_arxiv/"+f, "w") as out:
        json.dump(tgt_text, out)
    
te = time.time()

In [84]:
ascores = []
for i,p in enumerate(papers):
    if i >= len(asummaries):
        break
    s = scorer.score(p["summary"], summaries[i][0])
    
    #simpler flat structure
    x = {
        "rouge1:precision":s["rouge1"][0],
        "rouge1:recall":s["rouge1"][1],
        "rouge1:fmeasure":s["rouge1"][2],
        "rouge2:precision":s["rouge2"][0],
        "rouge2:recall":s["rouge2"][1],
        "rouge2:fmeasure":s["rouge2"][2],
        "rougeL:precision":s["rougeL"][0],
        "rougeL:recall":s["rougeL"][1],
        "rougeL:fmeasure":s["rougeL"][2],
        "rougeLsum:precision":s["rougeLsum"][0],
        "rougeLsum:recall":s["rougeLsum"][1],
        "rougeLsum:fmeasure":s["rougeLsum"][2],
        
    }
    ascores.append(x)

In [85]:
ascores_df = pd.DataFrame(ascores)
ascores_df.describe()

,rouge1:precision,rouge1:recall,rouge1:fmeasure,rouge2:precision,rouge2:recall,rouge2:fmeasure,rougeL:precision,rougeL:recall,rougeL:fmeasure,rougeLsum:precision,rougeLsum:recall,rougeLsum:fmeasure
count,1002.000000,1002.000000,1002.000000,1002.000000,1002.000000,1002.000000,1002.000000,1002.000000,1002.000000,1002.000000,1002.000000,1002.000000
mean,0.683712,0.140633,0.217190,0.449862,0.093938,0.144174,0.589455,0.122371,0.188390,0.653081,0.133835,0.206797
std,0.302874,0.127867,0.157005,0.340083,0.121041,0.155497,0.304572,0.124171,0.154270,0.300739,0.124504,0.153503
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.500000,0.066336,0.118387,0.133333,0.019512,0.033946,0.360909,0.053070,0.093842,0.450403,0.060606,0.109631
50%,0.769231,0.111111,0.185499,0.400000,0.056259,0.096515,0.608696,0.088235,0.146924,0.722222,0.104707,0.177345
75%,0.937500,0.172733,0.281480,0.766176,0.115623,0.197734,0.875000,0.145739,0.236842,0.904762,0.166322,0.265256
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# SPLITTING PARAGRAPHS
# SPLITTING PARAGRAPHS
# SPLITTING PARAGRAPHS
# SPLITTING PARAGRAPHS
# SPLITTING PARAGRAPHS
# SPLITTING PARAGRAPHS

In [53]:
#load our 10k data into a dataframe
papers = []
filenames = [] #keep a reference for later transformations

abstracts2, bodys2, cits2 =[],[],[]

for root, dirs, files in os.walk(DATASET_PATH):
    for f in files:
        fn = root+"/"+f
        if "abstract" in fn:            
            in_file = open(fn, 'r')
            file = in_file.readlines()

            new_list = [''.join(file[i*4:(i+1)*4]) for i in range(int(len(file)/4))]
            list_no_n = [item.replace('\n','') for item in new_list]

            string = ''
            for item in list_no_n:
                string = string + item            
            abstracts2.append(string)
            
        elif "body" in fn:
            in_file = open(fn, 'r')
            file = in_file.readlines()

            new_list = [''.join(file[i*4:(i+1)*4]) for i in range(int(len(file)/4))]
            
            list_no_n = [item.replace('. \n','@@@@@').replace('\n','').replace('- ','').replace("\'", "") for item in new_list]
            #list_no_n = [item for item in new_list]
            string = ''
            for item in list_no_n:
                string = string + item            
            bodys2.append(string)
        
        elif "CS" in fn:
            in_file = open(fn, 'r')
            file = in_file.readlines()
            string = [''.join(file) for i in file]
            if len(string) > 0: 
                string = [''.join(file) for i in file][0].replace('\n','')
            cits2.append(string)
            
        else:
            pass
        
        
        
        
    #    with open(fn) as jsonfile:
    #        d = json.load(jsonfile)
    #    papers.append(d)
    #    filenames.append(f)

df2 = pd.DataFrame({'Abstract':abstracts2,'Body':bodys2,'Citations':cits2})
df2
        

,Abstract,Body,Citations
0,A major obstacle to the construction ofa pro...,Parallel texts have been used in a number of...,A compilation of parallel texts offered in a s...
1,This paper proposes a way to improve the tran...,"Recently, various dialogue translation syste...",We plan to improve the accuracy obtained so fa...
2,This paper describes an application of APE (t...,The purpose of the Atlas project is to enlarg...,Computer dialogue is now used at production st...
3,In this paper we describe an implemented fram...,In this paper we present a linguistically mot...,"From this viewpoint, research on paraphrasing ..."
4,"This paper reports on a large-scale, end-to- ...",One major goal of information extraction (IE)...,"For example, Aone and Ramos-Santacruz (2000) p..."
...,...,...,...
4268,We present in this paper an unsupervised meth...,Development of electronic morphological resou...,Next along the spectrum of orthographic simila...
4269,This paper presents an unsupervised method fo...,Choosing the correct translation of a content...,"For comparable corpora, previous bilingual sen..."
4270,In this paper we report on an unsupervised a...,In this paper we discuss a potential solutio...,Watkinson and Manandhar (1999) present an unsu...
4271,This paper presents results from a study comp...,In evaluating the state of technology for ext...,"It may be noted that ""correctly"" is a problema..."


In [93]:
df2.Body[0]

'  Parallel texts have been used in a number of studies in computational linguistics. Brown et al (1993) defined a series of probabilistic translation models for MT purposes. While people may question the effectiveness of using these models for a full-blown MT system, the models are certainly valuable for developing translation assistance tools. For example, we can use such a translation model to help complete target ext being drafted by a human translator (Langlais et al, 2000)@@@@@Another utilization is in cross-language information retrieval (CLIR) where queries have to be translated from one language to another language in which the documents are written. In CLIR, the quality requirement for translation is relatively low. For example, the syntactic aspect is irrelevant. Even if the translated word is not a true translation but is strongly related to the original query, it is still helpful. Therefore, CLIR is a suitable application for such a translation model@@@@@However, a major o

In [96]:
df2.Body[0]

'  Parallel texts have been used in a number of studies in computational linguistics. Brown et al (1993) defined a series of probabilistic translation models for MT purposes. While people may question the effectiveness of using these models for a full-blown MT system, the models are certainly valuable for developing translation assistance tools. For example, we can use such a translation model to help complete target ext being drafted by a human translator (Langlais et al, 2000)@@@@@Another utilization is in cross-language information retrieval (CLIR) where queries have to be translated from one language to another language in which the documents are written. In CLIR, the quality requirement for translation is relatively low. For example, the syntactic aspect is irrelevant. Even if the translated word is not a true translation but is strongly related to the original query, it is still helpful. Therefore, CLIR is a suitable application for such a translation model@@@@@However, a major o

In [40]:

src_text = [
     """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
]

model_name = 'google/pegasus-xsum'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

In [79]:

src_text = [
     """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
]

model_name = 'google/pegasus-arxiv'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
amodel = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = amodel.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

In [90]:
summaries = []
k=0
for i in range(len(df2.Body)):
    word_sum = []
    string = df2.Body[i].split('@@@@@')
    string.append(df.Citations[i])    
    k+=1
    indicator = 0
    for sent in range(len(string)):
        if indicator == 1:
            indicator = 0
        else:
            if len(string[sent]) > 250:
                batch = tokenizer(string[sent], truncation=True, padding='longest', return_tensors="pt").to(device)
                indicator = 0
            elif len(str(string[sent]+" "+string[sent+1])) > 250:
                try:
                    batch = tokenizer(str(string[sent]+" "+string[sent+1]), truncation=True, padding='longest', return_tensors="pt").to(device)
                    indicator = 1
                except:
                    batch = tokenizer(string[sent], truncation=True, padding='longest', return_tensors="pt").to(device)
                    indicator = 0
            else: 
                try:
                    batch = tokenizer(str(string[sent]+" "+string[sent+1]+" "+string[sent+2]), truncation=True, padding='longest', return_tensors="pt").to(device)
                    indicator = 1
                except:
                    try:
                        batch = tokenizer(str(string[sent]+" "+string[sent+1]), truncation=True, padding='longest', return_tensors="pt").to(device)
                        indicator = 1
                    except:
                        batch = tokenizer(string[sent], truncation=True, padding='longest', return_tensors="pt").to(device)
                        indicator = 0
                        
            try:
                translated = model.generate(**batch)
                tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
                word_sum.append(tgt_text)
            except:
                pass
            
    sum_this = ''
    for x in word_sum:
        sum_this = sum_this + x[0] + " "
    
    f = open('aan_data_xsum_summaries_of_each_paragraph_r2.csv','a')
    f.write(sum_this)
    f.write('\n') 
    f.close()
    
    batch = tokenizer(sum_this, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = amodel.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    print(tgt_text)
    
    f = open('aan_data_xsum_summary_ofsum_of_each_paragraph_r2.csv','a')
    f.write(tgt_text[0])
    f.write('\n') 
    f.close()
    
    summaries.append(tgt_text)
    if k % 3 == 1:
        print(k)


['the goal of this paper is to explore the use of parallel texts in machine translation ( ). use of parallel texts in machine translation ( ) <n> is a promising method to train translation models. <n> text mining is a method to search for large amounts of parallel texts on the web-based tool engines that may contain parallel pages. <n> is a method to search for large amounts of parallel texts on the web-based tool engines that may contain parallel pages. <n> is a method to search for large amounts of parallel texts on the web-based tool engines that may contain parallel pages. <n> is a method to search for large amounts of parallel texts on the web-based tool engines that may contain parallel pages. <n> is a method to search for large amounts of parallel texts on the web-based tool engines that may contain parallel pages. <n> is a method to search for large amounts of parallel texts on the web-based tool engines that may contain parallel pages. <n> is a method to search for large amoun

IndexError: list index out of range

In [84]:
word_sum

[['The goal of this paper is to explore the use of parallel texts in machine translation (MT).'],
 ['One use case of this paper is the translation of documents from one language to another.'],
 ['Text mining is a promising method to train translation models.'],
 ['PTMiner is a text mining system designed to search for large amounts of parallel text on the Web.'],
 ['PTMiner is a web-based tool that searches the web for candidate sites that may contain parallel pages.'],
 ['The following is a guide to search engines and how they work:'],
 ['In our series of letters from African journalists, film-maker and columnist Ahmedou Ould-Abdallah looks at the importance of anchor texts.'],
 ['When searching for English parallel texts on the web, it is advisable to look for links to English parallel texts that contain the word "anchor".'],
 ['We do not have any English-language Web sites for Chinese parallel texts.'],
 ['The following table shows the search results for the following Web sites:'],


In [89]:
string[len(word_sum)+2]

'Comparing HTML structures seems to be a sound way to evaluate candidate pairs since parallel pairs usually have similar HTML structures. However, we also noticed that parallel texts may have quite different HTML structures. One of the reasons is that the two files may be created using two HTML editors. For example, one may be used for English and another for Chinese, depending on the language handling capability of the editors. Therefore, caution is required when measuring structure difference numerically'

In [75]:
sum_this = ''
for x in word_sum:
    sum_this = sum_this + x[0] + " "
print(sum_this)

batch = tokenizer(sum_this, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text)

The goal of this paper is to explore the use of parallel texts in machine translation (MT). One use case of this paper is the translation of documents from one language to another. Text mining is a promising method to train translation models. PTMiner is a text mining system designed to search for large amounts of parallel text on the Web. PTMiner is a web-based tool that searches the web for candidate sites that may contain parallel pages. The following is a guide to search engines and how they work: An anchor text is a piece of text that, when clicked on, will take you to another linked page. When searching for a website in English, you may notice that some pages contain links to other websites. We do not have any English parallel texts on our website. The following table shows the search results for the following Web sites: This article describes how to search for parallel texts using the search engines. PTMiner uses a host crawler to crawl documents on a given host. In this tutoria

In [73]:
df.Citations[0]

'A compilation of parallel texts offered in a serviceable form is called a parallel corpus. Parallel corpora are very valuable resources in various fields of multilingual natural language processing such as statistical machinet ranslation(Brown et al. , 1990), cross-lingual IR (Chenand Nie, 2000), and construction of dictionary(Nagao, 1996).'

In [58]:
len(word_sum)

32

In [66]:
len(string[16])

115

In [64]:
word_sum[16]

["BBC News takes a look at some of the key findings from this year's EU referendum."]

In [77]:
string

[' In recent years, speech and natural anguage technologies have matured enough to enable the development ofspoken dialogue systems in limited domains. Most existing systems employ dialogue strategies pre-specified during the design phase of the dialogue manager without taking into account characteristics of actual dialogue interactions. More specifically, mixed initiative systems typically employ rules that specify conditions (generally based on local dialogue context) under which initiative may shift from one agent o the other. Previous research, on the other hand, has shown that changes in initiative strategies inhuman-human dialogues can be dynamically modeled in terms of characteristics of the user and of the on-going dialogue (Chu-Carroll and Brown, 1998) and that adaptability of initiative strategies in dialogue systems leads to better system performance (Litman and Pan, 1999). However, no previous dialogue system takes into account these dialogue characteristics or allows for i

In [78]:
word_sum

[['The aim of this paper is to develop a dialogue system that takes into account the characteristics of actual dialogue interactions.'],
 ['Dialogue systems are an important part of the cinemagoing experience.'],
 ['This paper presents an adaptive mixed initiative dialogue manager that dynamically adapts dialogue strategies during an interaction based on information that can be automatically detected from the dialogue.'],
 ['First, MIMIC had task initiative until utterance (12), after which it switched to an open-ended prompt.'],
 ['In the first section of this paper, we show how MIMICs can be used to improve the performance of spoken dialogue systems.'],
 ['In this paper, we present an evidential framework for model prediction during humanhuman dialogues.'],
 ['This table shows the level of support for the user initiative in the current urn.'],
 ['This paper presents a framework for understanding the cues that affect the distribution of initiative in a dialogue.'],
 ['We present a pre